In [1]:
import numpy as np     # faster calculations for numerical matrices

In [2]:
def WeightAdd(dist, demands, locs):               # calculate matrix of wspd's given locating at locs and col j
    W = np.zeros( (len(demands), len(demands)) )    # initialize W
    for i in range(len(demands)):                   # for every row in W
        for j in range(len(demands)):               # for every column in W (to calc W[i, j])
            minDist = dist[i, j] * demands[i]       # weighted shortest path dist (wspd) from j to i     
            for locIndex in locs:                   # are already built locations better to ship from to i?
                distLoc = dist[i, locIndex] * demands[i]    # calculate wspd from loc to i
                if distLoc < minDist:                       # if loc better to ship from than j
                    minDist = distLoc                       # replace with better wspd
            W[i, j] = minDist                               # set W[i, j] to smallest wspd
    return(W)                                      # return nxn (n = num of nodes) matrix of wspd

In [3]:
def Add(dist, demands, alpha, fixed, sub = False):    # fixed needs to be numpy array of fixed costs
    go = True           # condition to keep iterating until adding doesn't reduce cost
    locs = []           # python list of where locate facilities
    cost = 99999999     # something very large to get over-written first iteration
    origFixed = fixed   # since np array, changes to fixed do not change origFixed!
    while go:
        W = WeightAdd(dist, demands, locs)         # calculate W matrix
        Tots = fixed + alpha * W.sum(axis = 0)     # get total costs (objective value) for adding each facility
        minIndex = np.argmin(Tots)                 # get index of best objective value
        if Tots[minIndex] < cost:                  # if improves objective
            cost = Tots[minIndex]                  # update cost
            locs.append(minIndex)                  # update locations
            fixed = fixed + origFixed[minIndex]    # update array of fixed costs -> each entry is total fixed costs
                                                   # of locating at current locs list and adding facility at index
        else:
            go = False                             # did not find better solution, terminate with current
    if sub:   # if using Greedy Add with Substitution
        locs, cost = Sub(locs, cost, minIndex, fixed, origFixed, dist, demands, alpha)   # Call Sub function
    nodeNames = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']   # Define node names for locations
    return({"Locations":sorted([nodeNames[i] for i in locs]), "Cost":cost})    # return locations and obj. value

In [4]:
def Sub(locs, cost, minIndex, fixed, origFixed, dist, demands, alpha):
    # try other locations and see if do better
    nodeNames = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']
    lastIndex = minIndex    # not sure if need
    origLocs = locs[:]         # Create copy so that try to replace every original location
    for chosenLoc in origLocs:        # try to replace every original location
        newLocs = locs                       # create copy
        newLocs.remove(chosenLoc)                 # remove one of locations
        fixed = fixed - origFixed[chosenLoc]      # adjust fixed costs
        W = WeightAdd(dist, demands, newLocs)     # recalculate W
        Tots = fixed + alpha * W.sum(axis = 0)    # get obj values
        minIndex = np.argmin(Tots)                # find minimum
        cost = Tots[minIndex]                     # update cost
        locs.append(minIndex)                     # update locations
        fixed = fixed + origFixed[minIndex]       # update fixed costs
    return(locs, cost)

In [5]:
def WeightDrop(dist, demands, locs):
    W = np.zeros( (len(demands), len(demands)) )      # initialize W
    for i in range(len(demands)):                     # for every row in W
        for j in range(len(demands)):                 # for every column in W (to calc W[i, j])
            minDist = 9999999   # absurd so gets overwritten
            for locIndex in locs:                     # check wspd for every current location
                if locIndex != j:                     # but we are dropping column j from locations (if there)
                    distLoc = dist[i, locIndex] * demands[i]    # calculate wspd from locIndex to i
                    if distLoc < minDist:             # if lower wspd overwrite minDist
                        minDist = distLoc
            W[i, j] = minDist                         # set W[i, j] to smallest wspd
    return(W)               # return nxn (n = num of nodes) matrix of wspd

In [6]:
def Drop(dist, demands, alpha, fixed):    # Drop Heuristic
    go = True                             # Keep dropping facilities while solution improves
    locs = list(range(len(demands)))      # Start by locating at all facilities
    cost = fixed.sum()                    # Cost = sum of all fixed costs
    origFixed = fixed                     # since np array, changes to fixed do not change origFixed
    
    calcFixed = np.repeat([cost], len(fixed))         # these will be fixed costs for calculations
    for j in range(len(fixed)):                       # for every location
        calcFixed[j] = calcFixed[j] - origFixed[j]    # update calcFixed to not include cost of locating at j
    
    while go:                               
        W = WeightDrop(dist, demands, locs)               # Calculate wspd matrix
        Tots = calcFixed + alpha * W.sum(axis = 0)        # Calculate total cost of removing each facility
        minIndex = np.argmin(Tots)                        # find best one
        if Tots[minIndex] < cost:                # if better than previous solution
            cost = Tots[minIndex]                # update cost   
            locs.remove(minIndex)                # update locations
            calcFixed = calcFixed - origFixed[minIndex]    # update fixed costs since removed facility
            calcFixed[minIndex] = calcFixed[minIndex] + origFixed[minIndex]    # don't change for minIndex facility
        else:
            go = False
    nodeNames = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']    # Names of nodes
    return({"Locations":sorted([nodeNames[i] for i in locs]), "Cost":cost})     # return locations and obj value

In [7]:
# initialize shortest path distance matrix
spd = np.array([[ 0, 10, 16, 10, 12, 16],    # Row A
                [10,  0,  6,  7, 16, 13],    # Row B
                [16,  6,  0,  7, 16,  9],    # Row C
                [10,  7,  7,  0,  9,  6],    # Row D
                [12, 16, 16,  9,  0,  9],    # Row E
                [16, 13,  9,  6,  9,  0]])   # Row F

# demand vector
dem = np.array([8, 10, 6, 5, 9, 10])

# Fixed costs vector
fc = np.array([40, 100, 50, 115, 70, 100])

# alpha value
a = 1

In [8]:
Add(spd, dem, a, fc)

{'Locations': ['A', 'C', 'D', 'E'], 'Cost': 395.0}

In [9]:
Drop(spd, dem, a, fc)

{'Locations': ['A', 'C', 'E'], 'Cost': 345.0}

In [10]:
Add(spd, dem, a, fc, sub = True)  # Should have same num of nodes as original Add

{'Locations': ['A', 'C', 'E', 'F'], 'Cost': 350.0}

In [11]:
# initialize shortest path distance matrix
                 #A  #B  #C  #D  #E  #F  #G  #H  #I  #J  #K  #L
spd = np.array([[ 0, 15, 37, 55, 24, 60, 18, 33, 48, 40, 58, 67],    # Row A
                [15,  0, 22, 40, 38, 52, 33, 48, 42, 55, 61, 61],    # Row B
                [37, 22,  0, 18, 16, 30, 41, 28, 20, 65, 39, 39],    # Row C
                [55, 40, 18,  0, 34, 12, 59, 46, 24, 62, 43, 34],    # Row D
                [24, 38, 16, 34,  0, 36, 25, 12, 24, 47, 37, 43],    # Row E
                [60, 52, 30, 12, 36,  0, 57, 42, 12, 50, 31, 22],    # Row F
                [18, 33, 41, 59, 25, 57,  0, 15, 45, 22, 40, 61],    # Row G
                [33, 48, 28, 46, 12, 42, 15,  0, 30, 37, 25, 46],    # Row H
                [48, 42, 20, 24, 24, 12, 45, 30,  0, 38, 19, 19],    # Row I
                [40, 55, 65, 62, 47, 50, 22, 37, 38,  0, 19, 40],    # Row J
                [58, 61, 39, 43, 37, 31, 40, 25, 19, 19,  0, 21],    # Row K
                [67, 61, 39, 34, 43, 22, 61, 46, 19, 40, 21,  0]])   # Row L  

# demand vector
                #A  #B  #C  #D  #E  #F  #G  #H  #I  #J  #K  #L
dem = np.array([15, 10, 12, 18,  5, 24, 11, 16, 13, 22, 19, 20])

# Fixed costs vector
                #A   #B   #C   #D   #E   #F   #G   #H   #I   #J   #K   #L
fc = np.array([100, 200, 130, 150, 225, 175, 190, 210, 165, 230, 125, 215])

# alpha value
a = 0.35

In [12]:
Add(spd, dem, a, fc)

{'Locations': ['A', 'D', 'G', 'I', 'K'], 'Cost': 1364.1999999999998}

In [13]:
Drop(spd, dem, a, fc)

{'Locations': ['A', 'C', 'F', 'K'], 'Cost': 1243.3}

In [14]:
Add(spd, dem, a, fc, sub = True)

{'Locations': ['A', 'C', 'F', 'H', 'K'], 'Cost': 1294.75}